# Modkit dmr - Replica 1 vs Replica 2b of CRoff day 35 data
## Use my Filtered Reads

Based on:
https://nanoporetech.github.io/modkit/intro_dmr.html#perform-differential-methylation-scoring

Select kernal: dimelo_v2_modkit_parsing

In [14]:
from datetime import datetime
def current_time():
    """Returns the current date and time as a formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
    
print("Current Date and Time:", current_time())

Current Date and Time: 2025-11-10 14:59:44


Preparing the input data
The inputs to all modkit dmr commands are two or more bedMethyl files (created by modkit pileup) that have been compressed with bgzip and indexed with tabix. An example of how to generate the input data is shown below:


ref=grch38.fasta
threads=32

norm=normal_sample.bam
norm_pileup=normal_pileup.bed

modkit pileup ${norm} ${norm_pileup} \
  --cpg \
  --ref ${ref} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${norm_pileup}
tabix -p bed ${norm_pileup}.gz

# pileup and compression can also be done in one step
tumor=tumor_sample.bam
tumor_pileup=tumor_pileup.bed.gz

modkit pileup ${tumor} - \
  --cpg \
  --ref ${ref} \
  --threads ${threads} \
  --log-filepath log.txt | ${bgzip} -c > ${tumor_pileup}

tabix -p bed ${tumor_pileup}

In [15]:
%%bash
echo "hello"

hello


# Use the NEW modkit latest installed version in ipython kernel modkit_new

In [16]:
# ! python3 -m ipykernel install --user --name=modkit_new --display-name "modkit_new Python"
# ! which modkit

In [17]:
import os
os.environ["PATH"] = "/home/michalula/.cargo/bin:" + os.environ["PATH"]
! which modkit
! modkit --version

/home/michalula/.cargo/bin/modkit
modkit 0.5.1


In [18]:
# ! modkit

In [19]:
! modkit --version 

modkit 0.5.1


In [20]:
1+1

2

In [21]:
import os
import pandas as pd

def load_pileup_bed(bed_path):
    # bed_path = existing[0]
    print("Reading bedMethyl file:", bed_path)

    # bedMethyl column names (18 columns as provided)
    colnames = [
        "chrom", "start", "end", "mod_code", "score", "strand",
        "start2", "end2", "color",
        "Nvalid_cov", "percent_modified", "Nmod", "Ncanonical",
        "Nother_mod", "Ndelete", "Nfail", "Ndiff", "Nnocall"
    ]

    # Configure dtypes where reasonable
    dtypes = {
        "chrom": str,
        "start": "Int64",
        "end": "Int64",
        "mod_code": str,
        "score": "Int64",
        "strand": str,
        "start2": "Int64",
        "end2": "Int64",
        "color": str,
        "Nvalid_cov": "Int64",
        "percent_modified": float,
        "Nmod": "Int64",
        "Ncanonical": "Int64",
        "Nother_mod": "Int64",
        "Ndelete": "Int64",
        "Nfail": "Int64",
        "Ndiff": "Int64",
        "Nnocall": "Int64"
    }

    compression = "gzip" if bed_path.endswith(".gz") else None

    # Read file (headerless BED-like table). If file has extra columns, keep them with automatic numeric conversion below.
    df = pd.read_csv(
        bed_path,
        sep="\t",
        header=None,
        comment="#",
        names=colnames,
        dtype=dtypes,
        compression=compression,
        engine="python",
        na_values=[".", "NA", ""],
        keep_default_na=True
    )

    # If file contained more than 18 columns, pandas assigned remaining data to extra columns named like col_18, col_19...
    # Ensure numeric conversion for numeric-like columns
    for c in df.columns:
        if df[c].dtype == object:
            # try safe numeric conversion where appropriate
            try:
                df[c] = pd.to_numeric(df[c], errors="ignore")
            except Exception:
                pass

    print("Loaded DataFrame shape:", df.shape)
    display(df.head())
    return df


In [22]:
import os
from IPython.display import display, HTML
from plotly import express as px
from plotly import graph_objects as go

# ! python3 -m pip install plotly
# ! python3 -m pip install matplotlib
# ! python3 -m pip install nbformat>=4.2.0

def plot_pileup_roi_df(df_roi, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    # ensure numeric types for plotting
    df_roi['pos'] = df_roi['start'].astype(int)
    df_roi['percent_modified'] = df_roi['percent_modified'].astype(float)
    df_roi['Nvalid_cov'] = df_roi['Nvalid_cov'].astype(int)
    df_roi['Nmod'] = df_roi['Nmod'].astype(int)
    df_roi['Ncanonical'] = df_roi['Ncanonical'].astype(int)

    # Scatter: genomic position vs percent modified (point size = coverage)
    fig1 = px.scatter(
        df_roi,
        x='pos',
        y='percent_modified',
        color='strand',
        size='Nvalid_cov',
        hover_data=['Nvalid_cov','Nmod','Ncanonical','Nother_mod','Nnocall'],
        title='Percent modified across ROI (size = Nvalid_cov)',
        height=500
    )
    fig1.update_layout(xaxis_title='Genomic position (start)', yaxis_title='Percent modified')
    fig1.show()
    # fig1.write_html(os.path.join(out_dir, "roi_percent_modified_scatter.html"), include_plotlyjs='cdn')

    # Histogram: coverage distribution
    fig2 = px.histogram(
        df_roi,
        x='Nvalid_cov',
        nbins=40,
        title='Distribution of Nvalid_cov (coverage) in ROI',
        height=400
    )
    fig2.update_layout(xaxis_title='Nvalid_cov', yaxis_title='Count')
    fig2.show()
    # fig2.write_html(os.path.join(out_dir, "roi_nvalidcov_hist.html"), include_plotlyjs='cdn')

    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked)
    topn = 30
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(name='Nmod', x=df_top['label'], y=df_top['Nmod']))
    fig3.add_trace(go.Bar(name='Ncanonical', x=df_top['label'], y=df_top['Ncanonical']))
    fig3.update_layout(barmode='stack', title=f'Top {topn} sites by percent_modified (stacked Nmod / Ncanonical)',
                    xaxis_title='position:strand', yaxis_title='reads', height=520)
    fig3.show()
    # fig3.write_html(os.path.join(out_dir, "roi_top_sites_stacked_counts.html"), include_plotlyjs='cdn')

    # Print simple summaries
    print("ROI rows:", df_roi.shape[0])
    print("Percent modified: median={:.2f}, mean={:.2f}".format(df_roi['percent_modified'].median(), df_roi['percent_modified'].mean()))
    print("Coverage (Nvalid_cov): min={}, median={}, max={}".format(df_roi['Nvalid_cov'].min(), df_roi['Nvalid_cov'].median(), df_roi['Nvalid_cov'].max()))

    # Display first rows table for quick inspection
    display(HTML(df_roi.head(20).to_html(index=False)))


    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked)
    topn = 274
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(name='Nmod', x=df_top['label'], y=df_top['Nmod']))
    fig3.add_trace(go.Bar(name='Ncanonical', x=df_top['label'], y=df_top['Ncanonical']))
    fig3.update_layout(barmode='stack', title=f'Top {topn} sites by percent_modified (stacked Nmod / Ncanonical)',
                    xaxis_title='position:strand', yaxis_title='reads', height=520)
    fig3.show()
    # fig3.write_html(os.path.join(out_dir, "roi_top_sites_stacked_counts.html"), include_plotlyjs='cdn')

    # Print simple summaries
    print("ROI rows:", df_roi.shape[0])
    print("Percent modified: median={:.2f}, mean={:.2f}".format(df_roi['percent_modified'].median(), df_roi['percent_modified'].mean()))
    print("Coverage (Nvalid_cov): min={}, median={}, max={}".format(df_roi['Nvalid_cov'].min(), df_roi['Nvalid_cov'].median(), df_roi['Nvalid_cov'].max()))

    # Display first rows table for quick inspection
    display(HTML(df_roi.head(20).to_html(index=False)))

    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    topn = 30
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig4 = go.Figure()
    fig4.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig4.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig4.update_layout(barmode='stack', title=f'Top {topn} sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig4.show()
    # fig4.write_html(os.path.join(out_dir, "roi_top_sites_stacked_percentage.html"), include_plotlyjs='cdn')     


    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    topn = 277
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig4 = go.Figure()
    fig4.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig4.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig4.update_layout(barmode='stack', title=f'Top {topn} sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig4.show()
    # fig4.write_html(os.path.join(out_dir, "roi_top_sites_stacked_percentage.html"), include_plotlyjs='cdn')     

    # Bar: Unsorted sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    df_top = df_roi.copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig5 = go.Figure()
    fig5.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig5.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig5.update_layout(barmode='stack', title=f'All sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig5.show()
    # fig5.write_html(os.path.join(out_dir, "roi_all_sites_stacked_percentage.html"), include_plotlyjs='cdn')    


    return df_top



In [23]:
! ls /home/michalula/data/ref_genomes/t2t_v2_0/

chm13v2.0.fa	  chm13v2.0.fa.fai		   haplotype_vcf
chm13v2.0.fa.amb  chm13v2.0.fa.pac		   up_chm13v2.0.fasta
chm13v2.0.fa.ann  chm13v2.0.fa.sa		   up_chm13v2.0.fasta.fai
chm13v2.0.fa.bwt  convert_to_uppercase_fasta.bash


# Pileups 
## for CRISPRoff filtered data for Day 35 
# Replica 2b:

In [12]:
! ls /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/

replica_1  replica_2b  replicas_compare


In [13]:
! ls /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output

CG_137_padded_reads_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.7_T2Tv2_0_chr1:206583354-206589854_2025-11-10_units_combined_numFWD78_numRVS167.npy
CG_137_padded_reads_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.995_T2Tv2_0_chr1:206583354-206589854_2025-11-10_units_combined_numFWD78_numRVS166.npy
extracted_reads
filtered_reads_overlap_MORE_than_0.9_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
filtered_reads_overlap_MORE_than_0.9_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam.bai
filtered_reads_overlap_MORE_than_0.9_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7

In [14]:
%%bash

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref= "/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"

threads=32

date_today="20251110"
# data_folder_path="/home/michalula/data/cas9_nanopore/data/20250908_nCATs_T_CRoff_Day_35/5mCG/to_t2t_v2_0/"
# CROFF_day35_bam=${data_folder_path}"sort_align_t2t_v2_0_trim_20250908_Day35_CROFF_Tcells_2Libraries_Minion_R9.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/"
# "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_dmr_pileup/"
mkdir ${pileup_data_folder_path}

# mC > 99.5% = 0995 filtered 
# filtered_CROFF_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# mC >70% = 07 filtered 
repl2b_filtered_CROFF_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# "filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
repl2b_pileup_CROFF_day35_bed=${pileup_data_folder_path}${date_today}"_repl2b""_filtered_mC07""_pileup_CROFF_Day35_Tcells.bed"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_CROFF_Day28/mergered_outputs/pileup_sort_merge_sort_align_t2t_v1_1_trim_20250721_nCATs_Tcells_CROFF_Day28_minion_run2_day8.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# norm=normal_sample.bam
# norm_pileup=normal_pileup.bed

modkit pileup ${repl2b_filtered_CROFF_day35_bam} ${repl2b_pileup_CROFF_day35_bed} \
  --cpg \
  --ref ${ref_genome_fa} \
  --threads ${threads} \
  --log-filepath log.txt


bgzip -k ${repl2b_pileup_CROFF_day35_bed}
tabix -p bed ${repl2b_pileup_CROFF_day35_bed}.gz


printf '%s\n' "repl2b_filtered_CROFF_day35_bam: $repl2b_filtered_CROFF_day35_bam"
printf '%s\n' "repl2b_pileup_CROFF_day35_bed: $repl2b_pileup_CROFF_day35_bed"
cat "$repl2b_pileup_CROFF_day35_bed"


mkdir: cannot create directory ‘/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/’: File exists
> calculated chunk size: 48, interval size 100000, processing 4800000 positions concurrently
> filtering to only CpG motifs
> attempting to sample 10042 reads
> Using filter threshold 0.7988281 for C.
> Done, processed 286 rows. Processed ~245 reads and skipped zero reads.
[tabix] the index file exists. Please use '-f' to overwrite.


repl2b_filtered_CROFF_day35_bam: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/filtered_reads_overlap_MORE_than_0.9_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
repl2b_pileup_CROFF_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed
chr1	206583089	206583090	m	1	+	206583089	206583090	255,0,0	1	100.00	1	0	0	0	0	0	0
chr1	206583090	206583091	m	1	-	206583090	206583091	255,0,0	1	100.00	1	0	0	0	0	0	0
chr1	206583173	206583174	m	117	+	206583173	206583174	255,0,0	117	88.03	103	14	0	1	23	8	5
chr1	206583174	206583175	m	85	-	206583174	206583175	255,0,0	85	97.65	83	2	0	0	1	1	2
chr1	206583387	206583388	m	123	+	206583387	206583388	255,0,0	123	7

In [15]:
date_today="20251110"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=data_folder_path+"new_dmr_pileup/"
# mkdir ${pileup_data_folder_path}

# mC > 99.5% = 0995 filtered 
# filtered_CROFF_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# filtered_CROFF_day35_bam=data_folder_path+"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# mC >70% = 07 filtered 
# filtered_CROFF_day35_bam=data_folder_path+"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"

repl2b_filtered_CROFF_day35_bam=data_folder_path+"filtered_reads_overlap_MORE_than_0.9_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
repl2b_pileup_CROFF_day35_bed=pileup_data_folder_path+date_today+"_repl2b""_filtered_mC07""_pileup_CROFF_Day35_Tcells.bed"

repl2b_filtered_CROFF_day35_bam, repl2b_pileup_CROFF_day35_bed

('/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/filtered_reads_overlap_MORE_than_0.9_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam',
 '/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed')

In [16]:
repl2b_pileup_CROFF_day35_df = load_pileup_bed(repl2b_pileup_CROFF_day35_bed)
repl2b_pileup_CROFF_day35_df

Reading bedMethyl file: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed
Loaded DataFrame shape: (286, 18)


/tmp/ipykernel_306177/825547292.py:60: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
0,chr1,206583089,206583090,m,1,+,206583089,206583090,"255,0,0",1,100.00,1,0,0,0,0,0,0
1,chr1,206583090,206583091,m,1,-,206583090,206583091,"255,0,0",1,100.00,1,0,0,0,0,0,0
2,chr1,206583173,206583174,m,117,+,206583173,206583174,"255,0,0",117,88.03,103,14,0,1,23,8,5
3,chr1,206583174,206583175,m,85,-,206583174,206583175,"255,0,0",85,97.65,83,2,0,0,1,1,2
4,chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
0,chr1,206583089,206583090,m,1,+,206583089,206583090,"255,0,0",1,100.00,1,0,0,0,0,0,0
1,chr1,206583090,206583091,m,1,-,206583090,206583091,"255,0,0",1,100.00,1,0,0,0,0,0,0
2,chr1,206583173,206583174,m,117,+,206583173,206583174,"255,0,0",117,88.03,103,14,0,1,23,8,5
3,chr1,206583174,206583175,m,85,-,206583174,206583175,"255,0,0",85,97.65,83,2,0,0,1,1,2
4,chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,chr1,206589931,206589932,m,5,-,206589931,206589932,"255,0,0",5,100.00,5,0,0,0,0,3,0
282,chr1,206589955,206589956,m,12,+,206589955,206589956,"255,0,0",12,100.00,12,0,0,0,2,2,0
283,chr1,206589956,206589957,m,4,-,206589956,206589957,"255,0,0",4,100.00,4,0,0,0,0,0,1
284,chr1,206590032,206590033,m,16,+,206590032,206590033,"255,0,0",16,68.75,11,5,0,0,0,0,0


In [18]:
# repl2b_pileup_CROFF_day35_df

In [19]:
repl2b_pileup_CROFF_day35_df_stats = plot_pileup_roi_df(df_roi=repl2b_pileup_CROFF_day35_df, out_dir=pileup_data_folder_path)
repl2b_pileup_CROFF_day35_df_stats

ROI rows: 286
Percent modified: median=80.00, mean=73.59
Coverage (Nvalid_cov): min=1, median=86.0, max=153


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583089,206583090,m,1,+,206583089,206583090,"255,0,0",1,100.00,1,0,0,0,0,0,0,206583089
chr1,206583090,206583091,m,1,-,206583090,206583091,"255,0,0",1,100.00,1,0,0,0,0,0,0,206583090
chr1,206583173,206583174,m,117,+,206583173,206583174,"255,0,0",117,88.03,103,14,0,1,23,8,5,206583173
chr1,206583174,206583175,m,85,-,206583174,206583175,"255,0,0",85,97.65,83,2,0,0,1,1,2,206583174
chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6,206583387
chr1,206583388,206583389,m,70,-,206583388,206583389,"255,0,0",70,84.29,59,11,0,6,5,2,7,206583388
chr1,206583707,206583708,m,144,+,206583707,206583708,"255,0,0",144,92.36,133,11,0,2,2,3,4,206583707
chr1,206583708,206583709,m,74,-,206583708,206583709,"255,0,0",74,91.89,68,6,0,4,4,5,3,206583708
chr1,206583766,206583767,m,140,+,206583766,206583767,"255,0,0",140,90.71,127,13,0,1,2,7,5,206583766
chr1,206583767,206583768,m,76,-,206583767,206583768,"255,0,0",76,97.37,74,2,0,4,7,1,2,206583767


ROI rows: 286
Percent modified: median=80.00, mean=73.59
Coverage (Nvalid_cov): min=1, median=86.0, max=153


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583089,206583090,m,1,+,206583089,206583090,"255,0,0",1,100.00,1,0,0,0,0,0,0,206583089
chr1,206583090,206583091,m,1,-,206583090,206583091,"255,0,0",1,100.00,1,0,0,0,0,0,0,206583090
chr1,206583173,206583174,m,117,+,206583173,206583174,"255,0,0",117,88.03,103,14,0,1,23,8,5,206583173
chr1,206583174,206583175,m,85,-,206583174,206583175,"255,0,0",85,97.65,83,2,0,0,1,1,2,206583174
chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6,206583387
chr1,206583388,206583389,m,70,-,206583388,206583389,"255,0,0",70,84.29,59,11,0,6,5,2,7,206583388
chr1,206583707,206583708,m,144,+,206583707,206583708,"255,0,0",144,92.36,133,11,0,2,2,3,4,206583707
chr1,206583708,206583709,m,74,-,206583708,206583709,"255,0,0",74,91.89,68,6,0,4,4,5,3,206583708
chr1,206583766,206583767,m,140,+,206583766,206583767,"255,0,0",140,90.71,127,13,0,1,2,7,5,206583766
chr1,206583767,206583768,m,76,-,206583767,206583768,"255,0,0",76,97.37,74,2,0,4,7,1,2,206583767


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,...,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos,label,Ntotal,Nmod_perc,Ncanonical_perc
0,chr1,206583089,206583090,m,1,+,206583089,206583090,"255,0,0",1,...,0,0,0,0,0,206583089,206583089:+,1,100.000000,0.000000
1,chr1,206583090,206583091,m,1,-,206583090,206583091,"255,0,0",1,...,0,0,0,0,0,206583090,206583090:-,1,100.000000,0.000000
2,chr1,206583173,206583174,m,117,+,206583173,206583174,"255,0,0",117,...,0,1,23,8,5,206583173,206583173:+,117,88.034188,11.965812
3,chr1,206583174,206583175,m,85,-,206583174,206583175,"255,0,0",85,...,0,0,1,1,2,206583174,206583174:-,85,97.647059,2.352941
4,chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,...,0,1,17,8,6,206583387,206583387:+,123,79.674797,20.325203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,chr1,206589931,206589932,m,5,-,206589931,206589932,"255,0,0",5,...,0,0,0,3,0,206589931,206589931:-,5,100.000000,0.000000
282,chr1,206589955,206589956,m,12,+,206589955,206589956,"255,0,0",12,...,0,0,2,2,0,206589955,206589955:+,12,100.000000,0.000000
283,chr1,206589956,206589957,m,4,-,206589956,206589957,"255,0,0",4,...,0,0,0,0,1,206589956,206589956:-,4,100.000000,0.000000
284,chr1,206590032,206590033,m,16,+,206590032,206590033,"255,0,0",16,...,0,0,0,0,0,206590032,206590032:+,16,68.750000,31.250000


# Look at CpGs within out target ROI
T2T v2.0

First CG:
206583388,206583390

Last of selected 137 CGs in the ROI:

206589746,206589748 --CpG_137

=> here have each CG position separate: so have 137*2  = 276

In [ ]:
137*2, 277-5

In [20]:
repl2b_pileup_CROFF_day35_df

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
0,chr1,206583089,206583090,m,1,+,206583089,206583090,"255,0,0",1,100.00,1,0,0,0,0,0,0,206583089
1,chr1,206583090,206583091,m,1,-,206583090,206583091,"255,0,0",1,100.00,1,0,0,0,0,0,0,206583090
2,chr1,206583173,206583174,m,117,+,206583173,206583174,"255,0,0",117,88.03,103,14,0,1,23,8,5,206583173
3,chr1,206583174,206583175,m,85,-,206583174,206583175,"255,0,0",85,97.65,83,2,0,0,1,1,2,206583174
4,chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6,206583387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,chr1,206589931,206589932,m,5,-,206589931,206589932,"255,0,0",5,100.00,5,0,0,0,0,3,0,206589931
282,chr1,206589955,206589956,m,12,+,206589955,206589956,"255,0,0",12,100.00,12,0,0,0,2,2,0,206589955
283,chr1,206589956,206589957,m,4,-,206589956,206589957,"255,0,0",4,100.00,4,0,0,0,0,0,1,206589956
284,chr1,206590032,206590033,m,16,+,206590032,206590033,"255,0,0",16,68.75,11,5,0,0,0,0,0,206590032


In [21]:
repl2b_pileup_CROFF_day35_df[repl2b_pileup_CROFF_day35_df['start'] == 206583387]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
4,chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6,206583387


In [22]:
repl2b_pileup_CROFF_day35_df[repl2b_pileup_CROFF_day35_df['start'] == 206583388]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
5,chr1,206583388,206583389,m,70,-,206583388,206583389,"255,0,0",70,84.29,59,11,0,6,5,2,7,206583388


In [23]:
repl2b_pileup_CROFF_day35_df[repl2b_pileup_CROFF_day35_df['start'] == 206589746]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
277,chr1,206589746,206589747,m,73,-,206589746,206589747,"255,0,0",73,95.89,70,3,0,5,5,2,5,206589746


In [24]:
(279-5) / 2

137.0

In [25]:
repl2b_pileup_CROFF_day35_df_roi = repl2b_pileup_CROFF_day35_df.iloc[4:278, :]  # Display target region rows
print(repl2b_pileup_CROFF_day35_df_roi.shape,repl2b_pileup_CROFF_day35_df_roi.shape[0]/2)
repl2b_pileup_CROFF_day35_df_roi

(274, 19) 137.0


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
4,chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6,206583387
5,chr1,206583388,206583389,m,70,-,206583388,206583389,"255,0,0",70,84.29,59,11,0,6,5,2,7,206583388
6,chr1,206583707,206583708,m,144,+,206583707,206583708,"255,0,0",144,92.36,133,11,0,2,2,3,4,206583707
7,chr1,206583708,206583709,m,74,-,206583708,206583709,"255,0,0",74,91.89,68,6,0,4,4,5,3,206583708
8,chr1,206583766,206583767,m,140,+,206583766,206583767,"255,0,0",140,90.71,127,13,0,1,2,7,5,206583766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,chr1,206589213,206589214,m,81,-,206589213,206589214,"255,0,0",81,97.53,79,2,0,0,5,2,2,206589213
274,chr1,206589436,206589437,m,149,+,206589436,206589437,"255,0,0",149,95.30,142,7,0,0,2,2,2,206589436
275,chr1,206589437,206589438,m,86,-,206589437,206589438,"255,0,0",86,96.51,83,3,0,2,0,2,0,206589437
276,chr1,206589745,206589746,m,133,+,206589745,206589746,"255,0,0",133,99.25,132,1,0,6,4,2,10,206589745


In [ ]:
repl2b_pileup_CROFF_day35_df_roi_stats = plot_pileup_roi_df(df_roi=repl2b_pileup_CROFF_day35_df_roi, out_dir=pileup_data_folder_path)
repl2b_pileup_CROFF_day35_df_roi_stats

/tmp/ipykernel_306177/3750650593.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_306177/3750650593.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_306177/3750650593.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipyke

ROI rows: 274
Percent modified: median=79.68, mean=72.73
Coverage (Nvalid_cov): min=19, median=86.5, max=153


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6,206583387
chr1,206583388,206583389,m,70,-,206583388,206583389,"255,0,0",70,84.29,59,11,0,6,5,2,7,206583388
chr1,206583707,206583708,m,144,+,206583707,206583708,"255,0,0",144,92.36,133,11,0,2,2,3,4,206583707
chr1,206583708,206583709,m,74,-,206583708,206583709,"255,0,0",74,91.89,68,6,0,4,4,5,3,206583708
chr1,206583766,206583767,m,140,+,206583766,206583767,"255,0,0",140,90.71,127,13,0,1,2,7,5,206583766
chr1,206583767,206583768,m,76,-,206583767,206583768,"255,0,0",76,97.37,74,2,0,4,7,1,2,206583767
chr1,206584104,206584105,m,143,+,206584104,206584105,"255,0,0",143,95.80,137,6,0,1,1,9,1,206584104
chr1,206584105,206584106,m,78,-,206584105,206584106,"255,0,0",78,97.44,76,2,0,3,4,3,2,206584105
chr1,206584137,206584138,m,151,+,206584137,206584138,"255,0,0",151,99.34,150,1,0,0,4,0,0,206584137
chr1,206584138,206584139,m,64,-,206584138,206584139,"255,0,0",64,92.19,59,5,0,0,25,0,1,206584138


ROI rows: 274
Percent modified: median=79.68, mean=72.73
Coverage (Nvalid_cov): min=19, median=86.5, max=153


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,79.67,98,25,0,1,17,8,6,206583387
chr1,206583388,206583389,m,70,-,206583388,206583389,"255,0,0",70,84.29,59,11,0,6,5,2,7,206583388
chr1,206583707,206583708,m,144,+,206583707,206583708,"255,0,0",144,92.36,133,11,0,2,2,3,4,206583707
chr1,206583708,206583709,m,74,-,206583708,206583709,"255,0,0",74,91.89,68,6,0,4,4,5,3,206583708
chr1,206583766,206583767,m,140,+,206583766,206583767,"255,0,0",140,90.71,127,13,0,1,2,7,5,206583766
chr1,206583767,206583768,m,76,-,206583767,206583768,"255,0,0",76,97.37,74,2,0,4,7,1,2,206583767
chr1,206584104,206584105,m,143,+,206584104,206584105,"255,0,0",143,95.80,137,6,0,1,1,9,1,206584104
chr1,206584105,206584106,m,78,-,206584105,206584106,"255,0,0",78,97.44,76,2,0,3,4,3,2,206584105
chr1,206584137,206584138,m,151,+,206584137,206584138,"255,0,0",151,99.34,150,1,0,0,4,0,0,206584137
chr1,206584138,206584139,m,64,-,206584138,206584139,"255,0,0",64,92.19,59,5,0,0,25,0,1,206584138


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,...,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos,label,Ntotal,Nmod_perc,Ncanonical_perc
4,chr1,206583387,206583388,m,123,+,206583387,206583388,"255,0,0",123,...,0,1,17,8,6,206583387,206583387:+,123,79.674797,20.325203
5,chr1,206583388,206583389,m,70,-,206583388,206583389,"255,0,0",70,...,0,6,5,2,7,206583388,206583388:-,70,84.285714,15.714286
6,chr1,206583707,206583708,m,144,+,206583707,206583708,"255,0,0",144,...,0,2,2,3,4,206583707,206583707:+,144,92.361111,7.638889
7,chr1,206583708,206583709,m,74,-,206583708,206583709,"255,0,0",74,...,0,4,4,5,3,206583708,206583708:-,74,91.891892,8.108108
8,chr1,206583766,206583767,m,140,+,206583766,206583767,"255,0,0",140,...,0,1,2,7,5,206583766,206583766:+,140,90.714286,9.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,chr1,206589213,206589214,m,81,-,206589213,206589214,"255,0,0",81,...,0,0,5,2,2,206589213,206589213:-,81,97.530864,2.469136
274,chr1,206589436,206589437,m,149,+,206589436,206589437,"255,0,0",149,...,0,0,2,2,2,206589436,206589436:+,149,95.302013,4.697987
275,chr1,206589437,206589438,m,86,-,206589437,206589438,"255,0,0",86,...,0,2,0,2,0,206589437,206589437:-,86,96.511628,3.488372
276,chr1,206589745,206589746,m,133,+,206589745,206589746,"255,0,0",133,...,0,6,4,2,10,206589745,206589745:+,133,99.248120,0.751880


# CRoff day 35 - Replica 1:

In [27]:
! ls /home/michalula/data/cas9_nanopore/data/20250908_nCATs_T_CRoff_Day_35/5mCG/to_t2t_v2_0

20251109_full_data_pileup_CROFF_Day35_Tcells.bed
20251109_full_data_pileup_CROFF_Day35_Tcells.bed.gz
20251109_full_data_pileup_CROFF_Day35_Tcells.bed.gz.tbi
align_t2t_v2_0_trim_20250908_Day35_CROFF_Tcells_2Libraries_Minion_R9.dna_r9.4.1_e8_sup@v3.3.5mCG.bam
sort_align_t2t_v2_0_trim_20250908_Day35_CROFF_Tcells_2Libraries_Minion_R9.dna_r9.4.1_e8_sup@v3.3.5mCG.bam
sort_align_t2t_v2_0_trim_20250908_Day35_CROFF_Tcells_2Libraries_Minion_R9.dna_r9.4.1_e8_sup@v3.3.5mCG.bam.bai
summary_sort_align_t2t_v2_0_trim_20250908_Day35_CROFF_Tcells_2Libraries_Minion_R9.dna_r9.4.1_e8_sup@v3.3.5mCG.tsv


In [28]:
! ls /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1
# /analyze_single_reads/dimelo_v2_output/

analyze_single_reads


In [47]:
%%bash

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref= "/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"

threads=32

date_today="20251110"
# data_folder_path="/home/michalula/data/cas9_nanopore/data/20250908_nCATs_T_CRoff_Day_35/5mCG/to_t2t_v2_0/"
# CROFF_day35_bam=${data_folder_path}"sort_align_t2t_v2_0_trim_20250908_Day35_CROFF_Tcells_2Libraries_Minion_R9.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_dmr_pileup/"
mkdir ${pileup_data_folder_path}

# mC > 99.5% = 0995 filtered 
# filtered_CROFF_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# mC >70% = 07 filtered 
repl1_filtered_CROFF_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
repl1_pileup_CROFF_day35_bed=${pileup_data_folder_path}${date_today}"_repl1_filtered_mC07""_pileup_CROFF_Day35_Tcells.bed"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_CROFF_Day28/mergered_outputs/pileup_sort_merge_sort_align_t2t_v1_1_trim_20250721_nCATs_Tcells_CROFF_Day28_minion_run2_day8.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# norm=normal_sample.bam
# norm_pileup=normal_pileup.bed

modkit pileup ${repl1_filtered_CROFF_day35_bam} ${repl1_pileup_CROFF_day35_bed} \
  --cpg \
  --ref ${ref_genome_fa} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${repl1_pileup_CROFF_day35_bed}
tabix -p bed ${repl1_pileup_CROFF_day35_bed}.gz

printf '%s\n' "repl1_filtered_CROFF_day35_bam: $repl1_filtered_CROFF_day35_bam"
printf '%s\n' "repl1_pileup_CROFF_day35_bed: $repl1_pileup_CROFF_day35_bed"
cat "$repl1_pileup_CROFF_day35_bed"


mkdir: cannot create directory ‘/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/’: File exists
> calculated chunk size: 48, interval size 100000, processing 4800000 positions concurrently
> filtering to only CpG motifs
> attempting to sample 10042 reads
> Using filter threshold 0.7910156 for C.
> Done, processed 327 rows. Processed ~326 reads and skipped zero reads.


repl1_filtered_CROFF_day35_bam: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
repl1_pileup_CROFF_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed
chr1	206583089	206583090	m	4	+	206583089	206583090	255,0,0	4	75.00	3	1	0	0	0	0	0
chr1	206583090	206583091	m	2	-	206583090	206583091	255,0,0	2	100.00	2	0	0	0	0	0	0
chr1	206583173	206583174	m	143	+	206583173	206583174	255,0,0	143	90.91	130	13	0	6	25	16	6
chr1	206583174	206583175	m	118	-	206583174	206583175	255,0,0	118	94.92	112	6	0	1	2	0	5
chr1	206583387	206583388	m	171	+	206583387	206583388	255,0,0	171	76.61	131	4

## Pileup columns explore

In [30]:
date_today="20251110"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=data_folder_path+"new_dmr_pileup/"
# mkdir ${pileup_data_folder_path}

# mC > 99.5% = 0995 filtered 
# filtered_CROFF_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# filtered_CROFF_day35_bam=data_folder_path+"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# mC >70% = 07 filtered 
repl1_filtered_CROFF_day35_bam=data_folder_path+"repl1_filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
repl1_pileup_CROFF_day35_bed=pileup_data_folder_path+date_today+"_filtered_mC07""_pileup_CROFF_Day35_Tcells.bed"


In [ ]:
repl1_pileup_CROFF_day35_df = load_pileup_bed(repl1_pileup_CROFF_day35_bed)
repl1_pileup_CROFF_day35_df

Reading bedMethyl file: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_filtered_mC07_pileup_CROFF_Day35_Tcells.bed
Loaded DataFrame shape: (327, 18)


/tmp/ipykernel_306177/825547292.py:60: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead



,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
0,chr1,206583089,206583090,m,4,+,206583089,206583090,"255,0,0",4,75.00,3,1,0,0,0,0,0
1,chr1,206583090,206583091,m,2,-,206583090,206583091,"255,0,0",2,100.00,2,0,0,0,0,0,0
2,chr1,206583173,206583174,m,143,+,206583173,206583174,"255,0,0",143,90.91,130,13,0,6,25,16,6
3,chr1,206583174,206583175,m,118,-,206583174,206583175,"255,0,0",118,94.92,112,6,0,1,2,0,5
4,chr1,206583387,206583388,m,171,+,206583387,206583388,"255,0,0",171,76.61,131,40,0,0,16,5,7


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
0,chr1,206583089,206583090,m,4,+,206583089,206583090,"255,0,0",4,75.00,3,1,0,0,0,0,0
1,chr1,206583090,206583091,m,2,-,206583090,206583091,"255,0,0",2,100.00,2,0,0,0,0,0,0
2,chr1,206583173,206583174,m,143,+,206583173,206583174,"255,0,0",143,90.91,130,13,0,6,25,16,6
3,chr1,206583174,206583175,m,118,-,206583174,206583175,"255,0,0",118,94.92,112,6,0,1,2,0,5
4,chr1,206583387,206583388,m,171,+,206583387,206583388,"255,0,0",171,76.61,131,40,0,0,16,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,chr1,206597139,206597140,m,1,+,206597139,206597140,"255,0,0",1,100.00,1,0,0,0,0,0,0
323,chr1,206597440,206597441,m,1,+,206597440,206597441,"255,0,0",1,100.00,1,0,0,0,0,0,0
324,chr1,206598045,206598046,m,1,+,206598045,206598046,"255,0,0",1,100.00,1,0,0,0,0,0,0
325,chr1,206598626,206598627,m,1,+,206598626,206598627,"255,0,0",1,100.00,1,0,0,0,0,0,0


# Look at CpGs within out target ROI
T2T v2.0

First CG:
206583388,206583390

Last of selected 137 CGs in the ROI:

206589746,206589748 --CpG_137

=> here have each CG position separate: so have 137*2  = 276

In [32]:
137*2, 277-5

(274, 272)

In [33]:
repl1_pileup_CROFF_day35_df[repl1_pileup_CROFF_day35_df['start'] == 206583387]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
4,chr1,206583387,206583388,m,171,+,206583387,206583388,"255,0,0",171,76.61,131,40,0,0,16,5,7


In [34]:
repl1_pileup_CROFF_day35_df[repl1_pileup_CROFF_day35_df['start'] == 206583388]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
5,chr1,206583388,206583389,m,98,-,206583388,206583389,"255,0,0",98,83.67,82,16,0,4,12,6,7


In [35]:
repl1_pileup_CROFF_day35_df[repl1_pileup_CROFF_day35_df['start'] == 206589746]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
277,chr1,206589746,206589747,m,102,-,206589746,206589747,"255,0,0",102,98.04,100,2,0,13,2,3,7


In [36]:
(279-5) / 2

137.0

In [37]:
repl1_pileup_CROFF_day35_df_roi = repl1_pileup_CROFF_day35_df.iloc[4:278, :]  # Display target region rows
print(repl1_pileup_CROFF_day35_df_roi.shape, repl1_pileup_CROFF_day35_df_roi.shape[0]/2)
repl1_pileup_CROFF_day35_df_roi

(274, 18) 137.0


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
4,chr1,206583387,206583388,m,171,+,206583387,206583388,"255,0,0",171,76.61,131,40,0,0,16,5,7
5,chr1,206583388,206583389,m,98,-,206583388,206583389,"255,0,0",98,83.67,82,16,0,4,12,6,7
6,chr1,206583707,206583708,m,187,+,206583707,206583708,"255,0,0",187,93.58,175,12,0,1,3,6,2
7,chr1,206583708,206583709,m,106,-,206583708,206583709,"255,0,0",106,96.23,102,4,0,6,8,1,6
8,chr1,206583766,206583767,m,169,+,206583766,206583767,"255,0,0",169,87.57,148,21,0,3,2,20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,chr1,206589213,206589214,m,114,-,206589213,206589214,"255,0,0",114,95.61,109,5,0,4,5,1,3
274,chr1,206589436,206589437,m,190,+,206589436,206589437,"255,0,0",190,92.63,176,14,0,1,2,4,2
275,chr1,206589437,206589438,m,116,-,206589437,206589438,"255,0,0",116,96.55,112,4,0,1,5,1,4
276,chr1,206589745,206589746,m,172,+,206589745,206589746,"255,0,0",172,100.00,172,0,0,7,2,6,11


<!>
> Threshold of  0.7597656 for base C is low. Consider increasing the filter-percentile or specifying a higher threshold.
> Done, processed 11762972 rows. Processed ~129977 reads and skipped ~150 reads.

In [38]:
# Plot pileup_Unedit_day35_df_roi summary plots (use existing variables/imports in the notebook)
# Saves interactive HTMLs to pileup_data_folder_path and displays inline.

repl1_pileup_CROFF_day35_df_roi_stats = plot_pileup_roi_df(df_roi=repl1_pileup_CROFF_day35_df_roi, out_dir=pileup_data_folder_path)
repl1_pileup_CROFF_day35_df_roi_stats


/tmp/ipykernel_306177/3750650593.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_306177/3750650593.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_306177/3750650593.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipyke

ROI rows: 274
Percent modified: median=81.37, mean=73.62
Coverage (Nvalid_cov): min=18, median=121.0, max=197


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583387,206583388,m,171,+,206583387,206583388,"255,0,0",171,76.61,131,40,0,0,16,5,7,206583387
chr1,206583388,206583389,m,98,-,206583388,206583389,"255,0,0",98,83.67,82,16,0,4,12,6,7,206583388
chr1,206583707,206583708,m,187,+,206583707,206583708,"255,0,0",187,93.58,175,12,0,1,3,6,2,206583707
chr1,206583708,206583709,m,106,-,206583708,206583709,"255,0,0",106,96.23,102,4,0,6,8,1,6,206583708
chr1,206583766,206583767,m,169,+,206583766,206583767,"255,0,0",169,87.57,148,21,0,3,2,20,5,206583766
chr1,206583767,206583768,m,105,-,206583767,206583768,"255,0,0",105,94.29,99,6,0,1,13,1,7,206583767
chr1,206584104,206584105,m,175,+,206584104,206584105,"255,0,0",175,94.29,165,10,0,4,2,15,3,206584104
chr1,206584105,206584106,m,111,-,206584105,206584106,"255,0,0",111,95.50,106,5,0,2,2,4,8,206584105
chr1,206584137,206584138,m,194,+,206584137,206584138,"255,0,0",194,97.42,189,5,0,0,5,0,0,206584137
chr1,206584138,206584139,m,83,-,206584138,206584139,"255,0,0",83,85.54,71,12,0,0,44,0,0,206584138


ROI rows: 274
Percent modified: median=81.37, mean=73.62
Coverage (Nvalid_cov): min=18, median=121.0, max=197


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583387,206583388,m,171,+,206583387,206583388,"255,0,0",171,76.61,131,40,0,0,16,5,7,206583387
chr1,206583388,206583389,m,98,-,206583388,206583389,"255,0,0",98,83.67,82,16,0,4,12,6,7,206583388
chr1,206583707,206583708,m,187,+,206583707,206583708,"255,0,0",187,93.58,175,12,0,1,3,6,2,206583707
chr1,206583708,206583709,m,106,-,206583708,206583709,"255,0,0",106,96.23,102,4,0,6,8,1,6,206583708
chr1,206583766,206583767,m,169,+,206583766,206583767,"255,0,0",169,87.57,148,21,0,3,2,20,5,206583766
chr1,206583767,206583768,m,105,-,206583767,206583768,"255,0,0",105,94.29,99,6,0,1,13,1,7,206583767
chr1,206584104,206584105,m,175,+,206584104,206584105,"255,0,0",175,94.29,165,10,0,4,2,15,3,206584104
chr1,206584105,206584106,m,111,-,206584105,206584106,"255,0,0",111,95.50,106,5,0,2,2,4,8,206584105
chr1,206584137,206584138,m,194,+,206584137,206584138,"255,0,0",194,97.42,189,5,0,0,5,0,0,206584137
chr1,206584138,206584139,m,83,-,206584138,206584139,"255,0,0",83,85.54,71,12,0,0,44,0,0,206584138


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,...,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos,label,Ntotal,Nmod_perc,Ncanonical_perc
4,chr1,206583387,206583388,m,171,+,206583387,206583388,"255,0,0",171,...,0,0,16,5,7,206583387,206583387:+,171,76.608187,23.391813
5,chr1,206583388,206583389,m,98,-,206583388,206583389,"255,0,0",98,...,0,4,12,6,7,206583388,206583388:-,98,83.673469,16.326531
6,chr1,206583707,206583708,m,187,+,206583707,206583708,"255,0,0",187,...,0,1,3,6,2,206583707,206583707:+,187,93.582888,6.417112
7,chr1,206583708,206583709,m,106,-,206583708,206583709,"255,0,0",106,...,0,6,8,1,6,206583708,206583708:-,106,96.226415,3.773585
8,chr1,206583766,206583767,m,169,+,206583766,206583767,"255,0,0",169,...,0,3,2,20,5,206583766,206583766:+,169,87.573964,12.426036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,chr1,206589213,206589214,m,114,-,206589213,206589214,"255,0,0",114,...,0,4,5,1,3,206589213,206589213:-,114,95.614035,4.385965
274,chr1,206589436,206589437,m,190,+,206589436,206589437,"255,0,0",190,...,0,1,2,4,2,206589436,206589436:+,190,92.631579,7.368421
275,chr1,206589437,206589438,m,116,-,206589437,206589438,"255,0,0",116,...,0,1,5,1,4,206589437,206589437:-,116,96.551724,3.448276
276,chr1,206589745,206589746,m,172,+,206589745,206589746,"255,0,0",172,...,0,7,2,6,11,206589745,206589745:+,172,100.000000,0.000000


# dmr modkit CRoff vs Unedited (NT)

In [40]:
pileup_data_folder_path

'/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/'

In [42]:
repl1_pileup_CROFF_day35_bed

'/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_filtered_mC07_pileup_CROFF_Day35_Tcells.bed'

In [46]:
%%bash

pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/"

pileup_CROFF_day35_bed=${pileup_data_folder_path}"20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"

echo "pileup_CROFF_day35_bed: ${pileup_CROFF_day35_bed}.gz"

ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_CROFF_day35_bed}.gz"


chmod u+rwx "${pileup_data_folder_path}"
chmod u+rwx "${pileup_CROFF_day35_bed}.gz"


ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_CROFF_day35_bed}.gz"


pileup_CROFF_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed
20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz.tbi
drwxrwxr-x 2 michalula michalula 4096 Nov 10 12:58 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/
-rw-rw-r-- 1 michalula michalula 6434 Nov 10 12:58 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
drwxrwxr-x 2 michalula michalula 4096 Nov 10 12:58 /home/michalula/code/epiCa

In [2]:
%%bash

pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/"

pileup_CROFF_day35_bed=${pileup_data_folder_path}"20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"

echo "pileup_CROFF_day35_bed: ${pileup_CROFF_day35_bed}.gz"


ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_CROFF_day35_bed}.gz"


chmod u+rwx "${pileup_data_folder_path}"
chmod u+rwx "${pileup_CROFF_day35_bed}.gz"


ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_CROFF_day35_bed}.gz"


pileup_CROFF_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed
20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz.tbi
drwxrwxr-x 2 michalula michalula 4096 Nov 10 14:29 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/
-rwxrw-r-- 1 michalula michalula 7115 Nov 10 14:29 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
drwxrwxr-x 2 michalula michalula 4096 Nov 10 14:29 /home/michalula/code/epiCausality/

In [6]:
%%bash
 
# 3. Detecting differential modification at single base positions
# The modkit dmr pair command has the ability to score individual bases (e.g. differentially methylated CpGs). To run single-base analysis on one or more paired samples, simply omit the --regions (-r) option when running modkit dmr pair. When performing single-base analysis the likelihood ratio score and a MAP-based p-value are available. For details on the likelihood ratio score and the MAP-based p-value, see the scoring details section. For example the above command becomes:
date_today="20251110"

experiment_codition="repl1_vs_repl2_CRoff_day35"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/"
dmr_result=${dmr_output_path}${date_today}"_single_base_filtered_mC07_"${experiment_codition}".bed"

# pileup_Unedit_day28_bam="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# removed the @ from the file name
# pileup_Unedit_day28_bam="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam"
# pileup_Unedit_day28_bam="/home/michalula/code/epiCausality/epiCode/differential_methyl/pileups/t2t_v1_1/pileup_Unedit_Day28_Tcells_20250721.bam"
# pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/"
# pileup_Unedit_day35_bed=${pileup_data_folder_path}"20251109_filtered_mC07_pileup_NT_Day35_Tcells.bed"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/pileup_Unedit_Day28_Tcells_20250721.bam"

echo "pileup_Unedit_day35_bed: ${pileup_Unedit_day35_bed}.gz"
ls -l "${pileup_Unedit_day35_bed}.gz"

# repl1_pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/"
# repl1_pileup_CROFF_day35_bed=${pileup_data_folder_path}"20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"
repl1_pileup_CROFF_day35_bed="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"
echo "repl1_pileup_CROFF_day35_bed: ${repl1_pileup_CROFF_day35_bed}.gz"
ls -l "${repl1_pileup_CROFF_day35_bed}.gz"

# repl2b_pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/ansalyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/"
# repl2b_pileup_CROFF_day35_bed=${pileup_data_folder_path}"20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"
repl2b_pileup_CROFF_day35_bed="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"
echo "repl2b_pileup_CROFF_day35_bed: ${repl2b_pileup_CROFF_day35_bed}.gz"
ls -l "${repl2b_pileup_CROFF_day35_bed}.gz"


ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"

threads=32
 
cd ${dmr_output_path}

modkit dmr pair \
  -a ${repl1_pileup_CROFF_day35_bed}.gz \
  -b ${repl2b_pileup_CROFF_day35_bed}.gz \
  -o ${dmr_result} \
  --ref ${ref_genome_fa} \
  --base C \
  --threads ${threads} \
  --log-filepath dmr.log


echo "dmr_result: $dmr_result"
ls -lah $dmr_result

pileup_Unedit_day35_bed: .gz


ls: cannot access '.gz': No such file or directory


repl1_pileup_CROFF_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
-rwxrw-r-- 1 michalula michalula 7115 Nov 10 14:29 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
repl2b_pileup_CROFF_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz
-rwxrw-r-- 1 michalula michalula 6434 Nov 10 12:58 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_rep

bash: line 38: cd: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/: No such file or directory
> creating directory at "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output"
> reading reference FASTA at "/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
> 1 common sequence(s) between FASTA and both samples
> running single-site analysis
> using default prior, Beta(α: 0.55, β: 0.55)
> estimating max coverages from data
> sampled 327 a records and 286 b records, calculating max coverages for 95th percentile
> calculated max coverage for a: 183 and b: 145
> calculated max coverage 183 is greater than maximum allowed (100), setting to 100
> calculated max coverage 145 is greater than maximum allowed (100), setting to 100
> errors:
+--------------------------+-------+
| error                    | count |
+-----

dmr_result: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/20251110_single_base_filtered_mC07_repl1_vs_repl2_CRoff_day35.bed
-rw-rw-r-- 1 michalula michalula 53K Nov 10 14:55 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/20251110_single_base_filtered_mC07_repl1_vs_repl2_CRoff_day35.bed


In [7]:
%%bash

date_today="20251110"

experiment_codition="repl1_vs_repl2_CRoff_day35"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/"
dmr_result=${dmr_output_path}${date_today}"_single_base_filtered_mC07_"${experiment_codition}".bed"

echo "dmr_result: $dmr_result"
# ls -lah $dmr_result
ls -lah $dmr_output_path
# cat $dmr_result

dmr_result: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/20251110_single_base_filtered_mC07_repl1_vs_repl2_CRoff_day35.bed
total 64K
drwxrwxr-x 2 michalula michalula 4.0K Nov 10 14:55 .
drwxrwxr-x 4 michalula michalula 4.0K Nov 10 14:55 ..
-rw-rw-r-- 1 michalula michalula  53K Nov 10 14:55 20251110_single_base_filtered_mC07_repl1_vs_repl2_CRoff_day35.bed


In [8]:
pwd

'/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replicas_compare/repl_modkit_dmr/mc_07_filtered'

## modkit dmr explore output

In [9]:

date_today="20251110"

experiment_codition="repl1_vs_repl2_CRoff_day35"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/"
dmr_result=dmr_output_path+date_today+"_single_base_filtered_mC07_"+experiment_codition+".bed"


dmr_path=dmr_result
# "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/new_dmr_output/20251109_single_base_filtered_mC07_day35_CRoff_vs_Unedit.bed"
dmr_path

'/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/20251110_single_base_filtered_mC07_repl1_vs_repl2_CRoff_day35.bed'

In [10]:
dmr_path

'/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/20251110_single_base_filtered_mC07_repl1_vs_repl2_CRoff_day35.bed'

In [24]:
# Read DMR BED (robust to header/no-header) and assign canonical column names (uses existing vars: dmr_path, out_dir, date_today, pd, os)
canonical_cols = [
    "chrom", "start", "end", "name", "score", "strand",
    "samplea_counts", "samplea_total", "sampleb_counts", "sampleb_total",
    "samplea_percents", "sampleb_percents",
    "samplea_fraction_modified", "sampleb_fraction_modified",
    "map_pvalue", "effect_size",
    "cohen_h", "cohen_h_low", "cohen_h_high",
]
    # "balanced_map_pvalue", "balanced_effect_size"

# read file with header and fallback to header=None when headers look numeric or columns are unexpected
try:
    dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")

    # dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", engine="python") # , header=0
    # # heuristic: if too many numeric-looking column names, re-read as headerless
    # numeric_headers = sum(1 for c in dmr_df.columns if str(c).strip().isdigit())
    # if numeric_headers >= (len(dmr_df.columns) / 2) or dmr_df.shape[1] < 3:
    #     dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")
except Exception:
    dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")

# assign canonical names up to number of columns present, add generic names for extras
ncols = dmr_df.shape[1]
if ncols <= len(canonical_cols):
    dmr_df.columns = canonical_cols[:ncols]
else:
    extras = [f"col_{i}" for i in range(ncols - len(canonical_cols))]
    dmr_df.columns = canonical_cols + extras

# coerce obvious numeric columns to numeric where present
num_cols_to_try = [
    "start", "end", "score",
    "samplea_total", "sampleb_total",
    "samplea_fraction_modified", "sampleb_fraction_modified",
    "map_pvalue", "effect_size",
    "balanced_map_pvalue", "balanced_effect_size"
]
for c in num_cols_to_try:
    if c in dmr_df.columns:
        dmr_df[c] = pd.to_numeric(dmr_df[c], errors="coerce")

# ensure output directory exists and save parsed table (parquet preferred)
os.makedirs(dmr_output_path, exist_ok=True)
parsed_path = os.path.join(dmr_output_path, f"{date_today}_dmr_parsed.parquet")
try:
    dmr_df.to_parquet(parsed_path, index=False)
    print("Saved parquet:", parsed_path)
except Exception:
    csv_path = os.path.join(dmr_output_path, f"{date_today}_dmr_parsed.csv")
    dmr_df.to_csv(csv_path, index=False)
    print("Parquet not available, saved csv:", csv_path)

print("Loaded DMR:", dmr_path)
print("Assigned columns:", dmr_df.columns.tolist())
print("Shape:", dmr_df.shape)
dmr_df.head()

Parquet not available, saved csv: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/20251110_dmr_parsed.csv
Loaded DMR: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/20251110_single_base_filtered_mC07_repl1_vs_repl2_CRoff_day35.bed
Assigned columns: ['chrom', 'start', 'end', 'name', 'score', 'strand', 'samplea_counts', 'samplea_total', 'sampleb_counts', 'sampleb_total', 'samplea_percents', 'sampleb_percents', 'samplea_fraction_modified', 'sampleb_fraction_modified', 'map_pvalue', 'effect_size', 'cohen_h', 'cohen_h_low', 'cohen_h_high']
Shape: (286, 19)


,chrom,start,end,name,score,strand,samplea_counts,samplea_total,sampleb_counts,sampleb_total,samplea_percents,sampleb_percents,samplea_fraction_modified,sampleb_fraction_modified,map_pvalue,effect_size,cohen_h,cohen_h_low,cohen_h_high
0,chr1,206583089,206583090,.,-0.031253,+,m:3,4,m:1,1,m:75.00,m:100.00,0.750000,1.000000,1.0,-0.250000,-1.047198,-1.144109,3.238504
1,chr1,206583090,206583091,.,-0.277632,-,m:2,2,m:1,1,m:100.00,m:100.00,1.000000,1.000000,1.0,0.000000,0.000000,-2.400456,2.400456
2,chr1,206583173,206583174,.,-0.059251,+,m:130,143,m:103,117,m:90.91,m:88.03,0.909091,0.880342,1.0,0.030000,0.093876,-0.150452,0.338204
3,chr1,206583174,206583175,.,0.182148,-,m:112,118,m:83,85,m:94.92,m:97.65,0.949152,0.976471,1.0,-0.026471,-0.146898,-0.131936,0.425732
4,chr1,206583387,206583388,.,-0.148286,+,m:131,171,m:98,123,m:76.61,m:79.67,0.766082,0.796748,1.0,-0.030000,-0.074241,-0.157483,0.305965


In [25]:
import os
from IPython.display import display, HTML

# Visualize all columns from dmr_df and save interactive HTMLs to out_dir
import plotly.express as px
import plotly.graph_objects as go

out_dir = dmr_output_path
print("out_dir: ", out_dir)

os.makedirs(out_dir, exist_ok=True)

# Save a table summary
summary = dmr_df.describe(include='all').transpose()
summary_path = os.path.join(out_dir, f"{date_today}_dmr_column_summary.csv")
summary.to_csv(summary_path)

numcols = dmr_df.select_dtypes(include=['number']).columns.tolist()

def _safe_name(name):
    return str(name).replace(os.sep, "_").replace(" ", "_").replace("\t", "_")

# Per-column visualizations
for col in dmr_df.columns:
    safe = _safe_name(col)
    try:
        if col in numcols:
            # Histogram
            fig_h = px.histogram(dmr_df, x=col, nbins=80, title=f"Histogram: {col}")
            # fig_h.write_html(os.path.join(out_dir, f"{date_today}_dmr_hist_{safe}.html"), include_plotlyjs='cdn')
            fig_h.show()

            # Boxplot
            fig_b = px.box(dmr_df, y=col, points="outliers", title=f"Boxplot: {col}")
            # fig_b.write_html(os.path.join(out_dir, f"{date_today}_dmr_box_{safe}.html"), include_plotlyjs='cdn')
            fig_b.show()
        else:
            # Categorical / text: show top value counts (up to 50)
            vc = dmr_df[col].fillna("NA").astype(str).value_counts().head(50)
            if len(vc):
                fig_c = px.bar(x=vc.values[::-1], y=vc.index.astype(str)[::-1], orientation='h',
                               title=f"Top value counts: {col}", labels={'x':'count','y':col})
                fig_c.update_layout(yaxis={'categoryorder':'array','categoryarray':vc.index[::-1].astype(str).tolist()})
                # fig_c.write_html(os.path.join(out_dir, f"{date_today}_dmr_valcounts_{safe}.html"), include_plotlyjs='cdn')
                fig_c.show()
            else:
                # fallback: display empty info
                display(HTML(f"<b>{col}</b>: no values to plot"))
    except Exception as e:
        print(f"Skipped plotting column {col!r} due to error: {e}")

# Correlation heatmap for numeric columns
if len(numcols) >= 2:
    try:
        corr = dmr_df[numcols].corr()
        fig_corr = px.imshow(corr, text_auto=True, aspect="auto", title="Correlation matrix (numeric columns)")
        # fig_corr.write_html(os.path.join(out_dir, f"{date_today}_dmr_correlation_numeric.html"), include_plotlyjs='cdn')
        fig_corr.show()
    except Exception as e:
        print("Failed to create correlation heatmap:", e)

print("Saved summary:", summary_path)
print("Plots saved to:", out_dir)

out_dir:  /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/


Saved summary: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/20251110_dmr_column_summary.csv
Plots saved to: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/


In [26]:
# Select significant CG pairs from DMR results and plot them (new cell at index 69).
# Uses existing notebook variables: dmr_df (parsed modkit dmr), df_roi_stats (pileup ROI stats),
# plotly (px) and out_dir/dmr_folder_path for saving. Does not re-import modules.

# Parameters
pvalue_thresh = 0.05

# ensure numeric columns
dmr_df['map_pvalue'] = pd.to_numeric(dmr_df['map_pvalue'], errors='coerce')
dmr_df['effect_size'] = pd.to_numeric(dmr_df['effect_size'], errors='coerce')
dmr_df['samplea_fraction_modified'] = pd.to_numeric(dmr_df['samplea_fraction_modified'], errors='coerce')
dmr_df['sampleb_fraction_modified'] = pd.to_numeric(dmr_df['sampleb_fraction_modified'], errors='coerce')

# filter significant by MAP-based p-value
sig = dmr_df[dmr_df['map_pvalue'] <= pvalue_thresh].copy()

dmr_df['map_pval_less005'] = dmr_df['map_pvalue'] <= 0.05


# # restrict to ROI positions if df_roi_stats exists
# if 'df_roi_stats' in globals():
#     roi_positions = set(df_roi_stats['start'].astype(int).tolist())
#     sig = sig[sig['start'].isin(roi_positions)].copy()

# quick exit if none
if sig.shape[0] == 0:
    print(f"No significant CG pairs found in ROI at map_pvalue <= {pvalue_thresh}")
else:
    # add convenience cols
    sig['pos'] = sig['start'].astype(str)
    sig['a_perc'] = sig['samplea_fraction_modified'] * 100
    sig['b_perc'] = sig['sampleb_fraction_modified'] * 100
    sig['total_reads'] = sig.get('samplea_total', 0).fillna(0).astype(int) + sig.get('sampleb_total', 0).fillna(0).astype(int)

    # save a table of significant sites
    os.makedirs(out_dir, exist_ok=True)
    sig_table_path = os.path.join(out_dir, f"dmr_significant_p{pvalue_thresh:.3f}_roi.tsv")
    sig.to_csv(sig_table_path, sep='\t', index=False)
    print("Saved significant sites table:", sig_table_path)
    display(sig[['chrom','start','end','strand','map_pvalue','effect_size','a_perc','b_perc','total_reads']].reset_index(drop=True))

    # plot the map_pval_less005 distribution which corresponds to significant sites
    fig_mappval_hist = px.histogram(
        dmr_df,
        x='map_pval_less005',
        nbins=80,
        title=f"MAP-based p-value distribution (highlighting p <= {pvalue_thresh})",
        labels={'map_pval_less005':'MAP-based p-value'}
    )
    fig_mappval_hist.update_layout(height=520)
    mappval_hist_path = os.path.join(out_dir, f"dmr_map_pval_distribution.html")
    # fig_mappval_hist.write_html(mappval_hist_path, include_plotlyjs='cdn')
    fig_mappval_hist.show()
    # print("Saved MAP-based p-value distribution histogram:", mappval_hist_path)

    # plot the percent of significant sites where map_pvalue <= pvalue_thresh is colored red, others blue (color not working)
    # Check https://plotly.com/python/pie-charts/ for coloring instructions
    percent_significant = (sig.shape[0] / dmr_df.shape[0]) * 100
    fig_mappval_pie = px.pie(
        dmr_df,
        names=['Not Significant (p > {})'.format(pvalue_thresh), 'Significant (p <= {})'.format(pvalue_thresh)],
        values=[dmr_df.shape[0] - sig.shape[0],sig.shape[0]],
        title=f"Percentage of significant CGs (map_pvalue <= {pvalue_thresh}): {percent_significant:.2f}%",
        # color_discrete_map={'Not Significant (p > {})'.format(pvalue_thresh): 'blue',
        #                     'Significant (p <= {})'.format(pvalue_thresh): 'red'},
        # colors=['blue','red']
    )
    fig_mappval_pie.update_layout(height=520)
    mappval_pie_path = os.path.join(out_dir, f"dmr_map_pval_percentage.html")
    # fig_mappval_pie.write_html(mappval_pie_path, include_plotlyjs='cdn')
    fig_mappval_pie.show()
    # print("Saved MAP-based p-value percentage pie chart:", mappval_pie_path)


    # plot effect size distribution of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh
    fig_effectsize_hist = px.histogram(
        dmr_df,
        x='effect_size',
        nbins=80,
        title=f"Effect size distribution (highlighting significant sites with map_pvalue <= {pvalue_thresh})",
        color=(dmr_df['map_pvalue'] <= pvalue_thresh), 
        color_discrete_map={True: 'red', False: 'blue'},
    )
            # labels={'effect_size':'Effect size (A - B)'}

    fig_effectsize_hist.update_layout(height=520)
    effectsize_hist_path = os.path.join(out_dir, f"dmr_effect_size_distribution.html")
    # fig_effectsize_hist.write_html(effectsize_hist_path, include_plotlyjs='cdn')
    fig_effectsize_hist.show()
    # print("Saved effect size distribution histogram:", effectsize_hist_path)        

    # plot effect sizes of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh 
    # add color legend names as 'Significant: map_pvalue <= pvalue_thresh' and 'Not Significant: map_pvalue > pvalue_thresh'  
    fig_effectsize_scatter = px.scatter(
        dmr_df,
        x=dmr_df.index,
        y='effect_size',
        color_discrete_map={True: 'red', False: 'blue'},
        color=(dmr_df['map_pvalue'] <= pvalue_thresh), 
        labels={'effect_size':'Effect size (A - B)','index':'Index',
                'color':f'Significant: map_pvalue <= {pvalue_thresh}'},
        title=f"Effect sizes for all CGs (highlighting significant sites with map_pvalue <= {pvalue_thresh})",
    )
    fig_effectsize_scatter.update_layout(height=520)
    effectsize_scatter_path = os.path.join(out_dir, f"dmr_effect_size_scatter.html")
    # fig_effectsize_scatter.write_html(effectsize_scatter_path, include_plotlyjs='cdn')
    fig_effectsize_scatter.show()
    # print("Saved effect size scatter plot:", effectsize_scatter_path)       

    # bar plot effect sizes of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh 
    fig_effectsize_bar = px.bar(
        dmr_df,
        x=dmr_df.index,         
        y='effect_size',
        color=(dmr_df['map_pvalue'] <= pvalue_thresh),
        labels={'effect_size':'Effect size (A - B)','index':'Index',
                'color':f'Significant: map_pvalue <= {pvalue_thresh}'},
        color_discrete_map={True: 'red', False: 'blue'},
        title=f"Effect sizes for all CGs (n={len(dmr_df)}) (highlighting significant sites with map_pvalue <= {pvalue_thresh})",
    )
    fig_effectsize_bar.update_layout(height=520)
    effectsize_bar_path = os.path.join(out_dir, f"dmr_effect_size_bar.html")
    # fig_effectsize_bar.write_html(effectsize_bar_path, include_plotlyjs='cdn')
    fig_effectsize_bar.show()
    # print("Saved effect size bar plot:", effectsize_bar_path) 


    # Bar: effect size per position (without sorting)
    sig['label'] = sig['pos'] + ":" + sig['strand'].astype(str)
    fig_bar_unsorted = px.bar(
        sig,
        x='label',
        y='effect_size',        
        color='effect_size',
        hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        title=f"Effect size for significant CGs (n={len(sig)}) with map_pvalue <= {pvalue_thresh}",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
    fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
    bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
    # fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
    fig_bar_unsorted.show()
    # print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
        

    # Bar: effect size per position (sorted)
    sig_sorted = sig.sort_values('effect_size', ascending=False).copy()
    sig_sorted['label'] = sig_sorted['pos'] + ":" + sig_sorted['strand'].astype(str)
    fig_bar = px.bar(
        sig_sorted,
        x='label',
        y='effect_size',
        color='effect_size',
        hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        title=f"Effect size for significant CGs (n={len(sig_sorted)})",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
    fig_bar.update_layout(xaxis_tickangle=45, height=520)
    bar_path = os.path.join(out_dir, f"dmr_sig_effectsize_p{pvalue_thresh:.3f}.html")
    # fig_bar.write_html(bar_path, include_plotlyjs='cdn')
    fig_bar.show()
    # print("Saved effect-size bar plot:", bar_path)



    # Scatter: sample A vs sample B percent modified (size = total reads, color = effect size)
    fig_scatter = px.scatter(
        sig,
        x='a_perc',
        y='b_perc',
        color='effect_size',
        size='total_reads',
        hover_data=['pos','start','map_pvalue','effect_size','cohen_h'],
        title=f"Significant CGs (map_pvalue <= {pvalue_thresh}) — sample A vs B percent modified",
        labels={'a_perc':'Sample A % modified','b_perc':'Sample B % modified'}
    )
    fig_scatter.update_layout(height=520)
    scatter_path = os.path.join(out_dir, f"dmr_sig_scatter_p{pvalue_thresh:.3f}.html")
    # fig_scatter.write_html(scatter_path, include_plotlyjs='cdn')
    fig_scatter.show()
    # print("Saved scatter plot:", scatter_path)


Saved significant sites table: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/rel12_croff_dmr_output/dmr_significant_p0.050_roi.tsv


,chrom,start,end,strand,map_pvalue,effect_size,a_perc,b_perc,total_reads
0,chr1,206585798,206585799,-,0.001049,0.259857,88.888890,62.903225,143
1,chr1,206586304,206586305,-,0.041314,0.244663,48.275860,23.809524,100
2,chr1,206587106,206587107,-,0.004474,0.229167,29.166666,6.250000,120
3,chr1,206587119,206587120,-,0.026022,0.239837,57.317070,33.333334,133


# TODO: check
- are there really NO diffs between the reads selected with the mC > 70 and mC > 99.5% filtering ??

could be as the mC calles are automatically selected

and in the CRoff the auto threshold 
* in mC > 70 was to 0.79
> Using filter threshold 0.7910156 for C.
* in mC > 99.5 was to  0.79
> Using filter threshold 0.7910156 for C.


and in the Unediter the auto threshold 
* in mC > 70 was to 0.8496
> Using filter threshold 0.8496094 for C.
* in mC > 99.5 was to 0.849
> Using filter threshold 0.8496094 for C.


SAME per condition AUTOMATIC modkit filtering threshold per mC run were set

(NOT 0.995 and not 0.7)

# Unedited T cells Day 35

In [ ]:
! ls "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output"

In [ ]:
%%bash

date_today="20251109"
data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_dmr_pileup/"
mkdir ${pileup_data_folder_path}

# mC > 99.5% = 0995 filtered 
# filtered_Unedit_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# mC >70% = 07 filtered   
filtered_Unedit_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
pileup_Unedit_day35_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_NT_Day35_Tcells.bed"
cat "$pileup_Unedit_day35_bed"

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref="/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"

threads=32

modkit pileup ${filtered_Unedit_day35_bam} ${pileup_Unedit_day35_bed} \
  --cpg \
  --ref ${ref_genome_fa} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${pileup_Unedit_day35_bed}
tabix -p bed ${pileup_Unedit_day35_bed}.gz

printf '%s\n' "filtered_Unedit_day35_bam: $filtered_Unedit_day35_bam"
printf '%s\n' "pileup_Unedit_day35_bed: $pileup_Unedit_day35_bed"
cat "$pileup_Unedit_day35_bed"


In [ ]:
date_today="20251109"
data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/"
pileup_data_folder_path=data_folder_path+"new_dmr_pileup/"
# mkdir ${pileup_data_folder_path}

# filtered_Unedit_day35_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
pileup_Unedit_day35_bed=pileup_data_folder_path+date_today+"_filtered_mC07""_pileup_NT_Day35_Tcells.bed"
print("pileup_Unedit_day35_bed:", pileup_Unedit_day35_bed)

pileup_Unedit_day35_df = load_pileup_bed(pileup_Unedit_day35_bed)
pileup_Unedit_day35_df

In [ ]:
pileup_Unedit_day35_df[pileup_Unedit_day35_df['start'] == 206583388-1]

In [ ]:
pileup_Unedit_day35_df[pileup_Unedit_day35_df['start'] == 206583388]

In [ ]:
pileup_Unedit_day35_df[pileup_Unedit_day35_df['start'] == 206589746-1]

In [ ]:
pileup_Unedit_day35_df[pileup_Unedit_day35_df['start'] == 206589746]

In [ ]:
pileup_Unedit_day35_df_roi = pileup_Unedit_day35_df.iloc[3:277, :]  # Display target region rows
print(pileup_Unedit_day35_df_roi.shape,pileup_Unedit_day35_df_roi.shape[0]/2)
pileup_Unedit_day35_df_roi

In [ ]:
pileup_Unedit_day35_df_roi

In [ ]:
pileup_Unedit_day35_df_roi

In [ ]:
pileup_data_folder_path

In [ ]:

# data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/"
# pileup_data_folder_path=data_folder_path+"new_dmr_pileup/"
# # mkdir ${pileup_data_folder_path}

# # Plot pileup_Unedit_day35_df_roi summary plots (use existing variables/imports in the notebook)
# # Saves interactive HTMLs to pileup_data_folder_path and displays inline.
# out_dir = pileup_data_folder_path  # existing variable in the notebook

# df_roi = pileup_Unedit_day35_df_roi.copy()

df_roi_stats = plot_pileup_roi_df(df_roi=pileup_Unedit_day35_df_roi, out_dir=pileup_data_folder_path)
df_roi_stats


# dmr modkit CRoff vs Unedited (NT)

3. Detecting differential modification at single base positions
The modkit dmr pair command has the ability to score individual bases (e.g. differentially methylated CpGs). To run single-base analysis on one or more paired samples, simply omit the --regions (-r) option when running modkit dmr pair. When performing single-base analysis the likelihood ratio score and a MAP-based p-value are available. For details on the likelihood ratio score and the MAP-based p-value, see the scoring details section. For example the above command becomes:

dmr_result=single_base_haplotype_dmr.bed

modkit dmr pair \
  -a ${hp1_pileup}.gz \
  -b ${hp2_pileup}.gz \
  -o ${dmr_result} \
  --ref ${ref} \
  --base C \
  --threads ${threads} \
  --log-filepath dmr.log

In [ ]:
# ! ls "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG"

In [ ]:
# ! ls /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup@v3.3.5mCG.bam.gz

In [ ]:
# /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam.gz

In [ ]:
# %%bash
# ls -ld /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs
# ls -l /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam.gz

In [ ]:
%%bash

pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/"
pileup_Unedit_day35_bed=${pileup_data_folder_path}"20251109_filtered_mC07_pileup_NT_Day35_Tcells.bed"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/pileup_Unedit_Day28_Tcells_20250721.bam"

echo "pileup_Unedit_day35_bed: ${pileup_Unedit_day35_bed}.gz"

ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_Unedit_day35_bed}.gz"
# ls -l "${pileup_Unedit_day28_bam}.gz"

chmod u+rwx "${pileup_data_folder_path}"
chmod u+rwx "${pileup_Unedit_day35_bed}.gz"

# ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_Unedit_day35_bed}.gz"


In [ ]:
%%bash

# pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/pileup/"
# pileup_Unedit_day35_bam=${pileup_data_folder_path}"20251109_filtered_pileup_NT_Day35_Tcells.bam"
# # "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/pileup_Unedit_Day28_Tcells_20250721.bam"
pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/"
pileup_CROFF_day35_bed=${pileup_data_folder_path}"20251109_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"

echo "pileup_CROFF_day35_bed: ${pileup_CROFF_day35_bed}.gz"

ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_CROFF_day35_bed}.gz"
# ls -l "${pileup_Unedit_day28_bam}.gz"

chmod u+rwx "${pileup_data_folder_path}"
chmod u+rwx "${pileup_CROFF_day35_bed}.gz"


# ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_CROFF_day35_bed}.gz"


In [ ]:
# %%bash

#    chmod u+rwx "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/"
#    chmod u+rwx "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_CROFF_Day28/mergered_outputs/5mCG/to_t2t_v2_0/pileup_CROFF_Day28_Tcells_20250721.bam.gz"

In [ ]:
# %%bash

# ls -ld /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs
# ls -ld "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_CROFF_Day28/mergered_outputs/5mCG/to_t2t_v2_0/pileup_CROFF_Day28_Tcells_20250721.bam.gz"

In [ ]:
# ls /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam.gz

In [ ]:
%%bash
 
# 3. Detecting differential modification at single base positions
# The modkit dmr pair command has the ability to score individual bases (e.g. differentially methylated CpGs). To run single-base analysis on one or more paired samples, simply omit the --regions (-r) option when running modkit dmr pair. When performing single-base analysis the likelihood ratio score and a MAP-based p-value are available. For details on the likelihood ratio score and the MAP-based p-value, see the scoring details section. For example the above command becomes:
date_today="20251109"

experiment_codition="day35_CRoff_vs_Unedit"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/new_dmr_output/"
dmr_result=${dmr_output_path}${date_today}"_single_base_filtered_mC07_"${experiment_codition}".bed"

# pileup_Unedit_day28_bam="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# removed the @ from the file name
# pileup_Unedit_day28_bam="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam"
# pileup_Unedit_day28_bam="/home/michalula/code/epiCausality/epiCode/differential_methyl/pileups/t2t_v1_1/pileup_Unedit_Day28_Tcells_20250721.bam"

pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/"
pileup_Unedit_day35_bed=${pileup_data_folder_path}"20251109_filtered_mC07_pileup_NT_Day35_Tcells.bed"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/pileup_Unedit_Day28_Tcells_20250721.bam"

echo "pileup_Unedit_day35_bed: ${pileup_Unedit_day35_bed}.gz"
ls -l "${pileup_Unedit_day35_bed}.gz"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam"
# pileup_Unedit_day28_bam_gz="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam.gz"
  
# pileup_CROFF_day28_bam="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_CROFF_Day28/mergered_outputs/pileup_sort_merge_sort_align_t2t_v1_1_trim_20250721_nCATs_Tcells_CROFF_Day28_minion_run2_day8.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# pileup_CROFF_day28_bam="/home/michalula/code/epiCausality/epiCode/differential_methyl/pileups/t2t_v1_1/pileup_CROFF_Day28_Tcells_20250721.bam"
pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/"
pileup_CROFF_day35_bed=${pileup_data_folder_path}"20251109_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"

echo "pileup_CROFF_day35_bed: ${pileup_CROFF_day35_bed}.gz"
ls -l "${pileup_CROFF_day35_bed}.gz"

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref="/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"
threads=32
 
cd ${dmr_output_path}
# '/home/michalula/code/epiCausality/epiCode/differential_methyl'

modkit dmr pair \
  -a ${pileup_CROFF_day35_bed}.gz \
  -b ${pileup_Unedit_day35_bed}.gz \
  -o ${dmr_result} \
  --ref ${ref_genome_fa} \
  --base C \
  --threads ${threads} \
  --log-filepath dmr.log


echo "dmr_result: $dmr_result"
ls -lah $dmr_result

In [ ]:
%%bash

date_today="20251109"
experiment_codition="day35_CRoff_vs_Unedit"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/new_dmr_output/"
dmr_result=${dmr_output_path}"_"${date_today}"single_base_filtered_mC07_"${experiment_codition}".bed"

echo "dmr_result: $dmr_result"
# ls -lah $dmr_result
ls -lah $dmr_output_path
# cat $dmr_result

In [ ]:
pwd

## modkit dmr explore output

The full table when performing single-site analysis with equal numbers of samples in groups, when running modkit dmr pair, will have the following schema:

column	name	description	type
1	chrom	name of reference sequence from bedMethyl input samples	str
2	start position	0-based start position, from --regions argument	int
3	end position	0-based exclusive end position, from --regions argument	int
4	name	name column from --regions BED, or chr:start-stop if absent, "." for single sites	str
5	score	difference score, more positive values have increased difference	float
6	strand	strand for the region or single-base position	str
7	samplea counts	counts of each base modification in the region, comma-separated, for sample A	str
8	samplea total	total number of base modification calls in the region, including unmodified, for sample A	int
9	sampleb counts	counts of each base modification in the region, comma-separated, for sample B	str
10	sampleb total	total number of base modification calls in the region, including unmodified, for sample B	int
11	samplea percents	percent of calls for each base modification in the region, comma-separated, for sample A	str
12	sampleb percents	percent of calls for each base modification in the region, comma-separated, for sample B	str
13	samplea fraction modified	fraction modification (of any kind) in sample A	float
14	sampleb fraction modified	fraction modification (of any kind) in sample B	float
15	MAP-based p-value	ratio of the posterior probability of observing the effect size over zero effect size	float
16	effect size	percent modified in sample A (col 12) minus percent modified in sample B (col 13)	float
<!-- 17	balanced MAP-based p-value	MAP-based p-value when all replicates are balanced	float
18	balanced effect size	effect size when all replicates are balanced	float -->


17	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
18	cohen_h_low	95% confidence interval lower bound	float
19	cohen_h_high	95% confidence interval upper bound	float

<!-- Differential methylation output format
The output from modkit dmr pair (and for each pairwise comparison with modkit dmr multi) is (roughly) a BED file with the following schema: -->
<!-- 
column	name	description	type
        1	chrom	name of reference sequence from bedMethyl input samples	str
        2	start position	0-based start position, from --regions argument	int
        3	end position	0-based exclusive end position, from --regions argument	int
        4	name	name column from --regions BED, or chr:start-stop if absent, "." for single sites	str
        5	score	difference score, more positive values have increased difference	float
        6	strand	strand for the region or single-base position	str
        7	samplea counts	counts of each base modification in the region, comma-separated, for sample A	str
        8	samplea total	total number of base modification calls in the region, including unmodified, for sample A	int
        9	sampleb counts	counts of each base modification in the region, comma-separated, for sample B	str
        10	sampleb total	total number of base modification calls in the region, including unmodified, for sample B	int
        11	samplea percents	percent of calls for each base modification in the region, comma-separated, for sample A	str
        12	sampleb percents	percent of calls for each base modification in the region, comma-separated, for sample B	str
        13	samplea fraction modified	fraction modification (of any kind) in sample A	float
        14	sampleb fraction modified	fraction modification (of any kind) in sample B	float
        15	MAP-based p-value	ratio of the posterior probability of observing the effect size over zero effect size	float
        16	effect size	percent modified in sample A (col 12) minus percent modified in sample B (col 13)	float
        17	balanced MAP-based p-value	MAP-based p-value when all replicates are balanced	float
        18	balanced effect size	effect size when all replicates are balanced	float
        19	pct_a_samples	percent of 'a' samples used in statistical test	float
        20	pct_b_samples	percent of 'b' samples used in statistical test	float
        21	per-replicate p-values	MAP-based p-values for matched replicate pairs	float
        22	per-replicate effect sizes	effect sizes matched replicate pairs	float
        23	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
        24	cohen_h_low	95% confidence interval lower bound	float
        25	cohen_h_high	95% confidence interval upper bound	float
        Columns 16-19 are only produced when multiple samples are provided, columns 20 and 21 are only produced when there is an equal number of 'a' and 'b' samples. When using multiple samples, it is possible that not every sample will have a modification fraction at a position. When this happens, the statistical test is still performed and the values of pct_a_samples and pct_b_samples reflect the percent of samples from each condition used in the test. 


     (15)	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
    (16)	cohen_h_low	95% confidence interval lower bound	float
    (17)	cohen_h_high	95% confidence interval upper bound	float
    
    n.b. Columns 15, 16, and 17 are present when the --regions option is passed, but these columns are on the right side of the table when performing single-site analysis (below). It is generally recommended to use the --header flag and standard CSV parsing to make sure the schema's between experiments are maintained.

When performing single-site analysis, the following additional columns are added:

column	name	description	type
Columns 20 and 21 have the replicate pairwise MAP-based p-values and effect sizes which are calculated based on their order provided on the command line. For example in the abbreviated command below:

In [ ]:
# # Path to the DMR BED output (adjust if you use a different file)
# date_today="20251109"
# experiment_codition="day35_CRoff_vs_Unedit"
# dmr_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/new_dmr_output/"
# dmr_path=dmr_folder_path+date_today+"_single_base_filtered_mC07_"+experiment_codition+".bed"

# out_dir = dmr_folder_path  # save artifacts alongside the DMR file
# os.makedirs(out_dir, exist_ok=True)

# # # Read the DMR file (header or no-header)
# # try:
# #     df = pd.read_csv(dmr_path, sep='	', comment='#', header=0, engine='python')
# #     if df.shape[1] < 3:
# #         df = pd.read_csv(dmr_path, sep='	', comment='#', header=None, engine='python')
# # except Exception as e:
# #     print("Failed to read DMR file:", e)
# #     raise

# df

In [ ]:
dmr_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_filtered/new_dmr_output/20251109_single_base_filtered_mC07_day35_CRoff_vs_Unedit.bed"

In [ ]:
dmr_path

In [ ]:
# Read DMR BED (robust to header/no-header) and assign canonical column names (uses existing vars: dmr_path, out_dir, date_today, pd, os)
canonical_cols = [
    "chrom", "start", "end", "name", "score", "strand",
    "samplea_counts", "samplea_total", "sampleb_counts", "sampleb_total",
    "samplea_percents", "sampleb_percents",
    "samplea_fraction_modified", "sampleb_fraction_modified",
    "map_pvalue", "effect_size",
    "cohen_h", "cohen_h_low", "cohen_h_high",
]
    # "balanced_map_pvalue", "balanced_effect_size"

# read file with header and fallback to header=None when headers look numeric or columns are unexpected
try:
    dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")

    # dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", engine="python") # , header=0
    # # heuristic: if too many numeric-looking column names, re-read as headerless
    # numeric_headers = sum(1 for c in dmr_df.columns if str(c).strip().isdigit())
    # if numeric_headers >= (len(dmr_df.columns) / 2) or dmr_df.shape[1] < 3:
    #     dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")
except Exception:
    dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")

# assign canonical names up to number of columns present, add generic names for extras
ncols = dmr_df.shape[1]
if ncols <= len(canonical_cols):
    dmr_df.columns = canonical_cols[:ncols]
else:
    extras = [f"col_{i}" for i in range(ncols - len(canonical_cols))]
    dmr_df.columns = canonical_cols + extras

# coerce obvious numeric columns to numeric where present
num_cols_to_try = [
    "start", "end", "score",
    "samplea_total", "sampleb_total",
    "samplea_fraction_modified", "sampleb_fraction_modified",
    "map_pvalue", "effect_size",
    "balanced_map_pvalue", "balanced_effect_size"
]
for c in num_cols_to_try:
    if c in dmr_df.columns:
        dmr_df[c] = pd.to_numeric(dmr_df[c], errors="coerce")

# ensure output directory exists and save parsed table (parquet preferred)
os.makedirs(out_dir, exist_ok=True)
parsed_path = os.path.join(out_dir, f"{date_today}_dmr_parsed.parquet")
try:
    dmr_df.to_parquet(parsed_path, index=False)
    print("Saved parquet:", parsed_path)
except Exception:
    csv_path = os.path.join(out_dir, f"{date_today}_dmr_parsed.csv")
    dmr_df.to_csv(csv_path, index=False)
    print("Parquet not available, saved csv:", csv_path)

print("Loaded DMR:", dmr_path)
print("Assigned columns:", dmr_df.columns.tolist())
print("Shape:", dmr_df.shape)
dmr_df.head()

In [ ]:
import os
from IPython.display import display, HTML

# Visualize all columns from dmr_df and save interactive HTMLs to out_dir
import plotly.express as px
import plotly.graph_objects as go

os.makedirs(out_dir, exist_ok=True)

# Save a table summary
summary = dmr_df.describe(include='all').transpose()
summary_path = os.path.join(out_dir, f"{date_today}_dmr_column_summary.csv")
summary.to_csv(summary_path)

numcols = dmr_df.select_dtypes(include=['number']).columns.tolist()

def _safe_name(name):
    return str(name).replace(os.sep, "_").replace(" ", "_").replace("\t", "_")

# Per-column visualizations
for col in dmr_df.columns:
    safe = _safe_name(col)
    try:
        if col in numcols:
            # Histogram
            fig_h = px.histogram(dmr_df, x=col, nbins=80, title=f"Histogram: {col}")
            # fig_h.write_html(os.path.join(out_dir, f"{date_today}_dmr_hist_{safe}.html"), include_plotlyjs='cdn')
            fig_h.show()

            # Boxplot
            fig_b = px.box(dmr_df, y=col, points="outliers", title=f"Boxplot: {col}")
            # fig_b.write_html(os.path.join(out_dir, f"{date_today}_dmr_box_{safe}.html"), include_plotlyjs='cdn')
            fig_b.show()
        else:
            # Categorical / text: show top value counts (up to 50)
            vc = dmr_df[col].fillna("NA").astype(str).value_counts().head(50)
            if len(vc):
                fig_c = px.bar(x=vc.values[::-1], y=vc.index.astype(str)[::-1], orientation='h',
                               title=f"Top value counts: {col}", labels={'x':'count','y':col})
                fig_c.update_layout(yaxis={'categoryorder':'array','categoryarray':vc.index[::-1].astype(str).tolist()})
                # fig_c.write_html(os.path.join(out_dir, f"{date_today}_dmr_valcounts_{safe}.html"), include_plotlyjs='cdn')
                fig_c.show()
            else:
                # fallback: display empty info
                display(HTML(f"<b>{col}</b>: no values to plot"))
    except Exception as e:
        print(f"Skipped plotting column {col!r} due to error: {e}")

# Correlation heatmap for numeric columns
if len(numcols) >= 2:
    try:
        corr = dmr_df[numcols].corr()
        fig_corr = px.imshow(corr, text_auto=True, aspect="auto", title="Correlation matrix (numeric columns)")
        # fig_corr.write_html(os.path.join(out_dir, f"{date_today}_dmr_correlation_numeric.html"), include_plotlyjs='cdn')
        fig_corr.show()
    except Exception as e:
        print("Failed to create correlation heatmap:", e)

print("Saved summary:", summary_path)
print("Plots saved to:", out_dir)

In [ ]:
# Select significant CG pairs from DMR results and plot them (new cell at index 69).
# Uses existing notebook variables: dmr_df (parsed modkit dmr), df_roi_stats (pileup ROI stats),
# plotly (px) and out_dir/dmr_folder_path for saving. Does not re-import modules.

# Parameters
pvalue_thresh = 0.05

# ensure numeric columns
dmr_df['map_pvalue'] = pd.to_numeric(dmr_df['map_pvalue'], errors='coerce')
dmr_df['effect_size'] = pd.to_numeric(dmr_df['effect_size'], errors='coerce')
dmr_df['samplea_fraction_modified'] = pd.to_numeric(dmr_df['samplea_fraction_modified'], errors='coerce')
dmr_df['sampleb_fraction_modified'] = pd.to_numeric(dmr_df['sampleb_fraction_modified'], errors='coerce')

# filter significant by MAP-based p-value
sig = dmr_df[dmr_df['map_pvalue'] <= pvalue_thresh].copy()

dmr_df['map_pval_less005'] = dmr_df['map_pvalue'] <= 0.05


# # restrict to ROI positions if df_roi_stats exists
# if 'df_roi_stats' in globals():
#     roi_positions = set(df_roi_stats['start'].astype(int).tolist())
#     sig = sig[sig['start'].isin(roi_positions)].copy()

# quick exit if none
if sig.shape[0] == 0:
    print(f"No significant CG pairs found in ROI at map_pvalue <= {pvalue_thresh}")
else:
    # add convenience cols
    sig['pos'] = sig['start'].astype(str)
    sig['a_perc'] = sig['samplea_fraction_modified'] * 100
    sig['b_perc'] = sig['sampleb_fraction_modified'] * 100
    sig['total_reads'] = sig.get('samplea_total', 0).fillna(0).astype(int) + sig.get('sampleb_total', 0).fillna(0).astype(int)

    # save a table of significant sites
    os.makedirs(out_dir, exist_ok=True)
    sig_table_path = os.path.join(out_dir, f"dmr_significant_p{pvalue_thresh:.3f}_roi.tsv")
    sig.to_csv(sig_table_path, sep='\t', index=False)
    print("Saved significant sites table:", sig_table_path)
    display(sig[['chrom','start','end','strand','map_pvalue','effect_size','a_perc','b_perc','total_reads']].reset_index(drop=True))

    # plot the map_pval_less005 distribution which corresponds to significant sites
    fig_mappval_hist = px.histogram(
        dmr_df,
        x='map_pval_less005',
        nbins=80,
        title=f"MAP-based p-value distribution (highlighting p <= {pvalue_thresh})",
        labels={'map_pval_less005':'MAP-based p-value'}
    )
    fig_mappval_hist.update_layout(height=520)
    mappval_hist_path = os.path.join(out_dir, f"dmr_map_pval_distribution.html")
    # fig_mappval_hist.write_html(mappval_hist_path, include_plotlyjs='cdn')
    fig_mappval_hist.show()
    # print("Saved MAP-based p-value distribution histogram:", mappval_hist_path)

    # plot the percent of significant sites where map_pvalue <= pvalue_thresh is colored red, others blue (color not working)
    # Check https://plotly.com/python/pie-charts/ for coloring instructions
    percent_significant = (sig.shape[0] / dmr_df.shape[0]) * 100
    fig_mappval_pie = px.pie(
        dmr_df,
        names=['Not Significant (p > {})'.format(pvalue_thresh), 'Significant (p <= {})'.format(pvalue_thresh)],
        values=[dmr_df.shape[0] - sig.shape[0],sig.shape[0]],
        title=f"Percentage of significant CGs (map_pvalue <= {pvalue_thresh}): {percent_significant:.2f}%",
        # color_discrete_map={'Not Significant (p > {})'.format(pvalue_thresh): 'blue',
        #                     'Significant (p <= {})'.format(pvalue_thresh): 'red'},
        # colors=['blue','red']
    )
    fig_mappval_pie.update_layout(height=520)
    mappval_pie_path = os.path.join(out_dir, f"dmr_map_pval_percentage.html")
    # fig_mappval_pie.write_html(mappval_pie_path, include_plotlyjs='cdn')
    fig_mappval_pie.show()
    # print("Saved MAP-based p-value percentage pie chart:", mappval_pie_path)


    # plot effect size distribution of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh
    fig_effectsize_hist = px.histogram(
        dmr_df,
        x='effect_size',
        nbins=80,
        title=f"Effect size distribution (highlighting significant sites with map_pvalue <= {pvalue_thresh})",
        color=(dmr_df['map_pvalue'] <= pvalue_thresh), 
        color_discrete_map={True: 'red', False: 'blue'},
    )
            # labels={'effect_size':'Effect size (A - B)'}

    fig_effectsize_hist.update_layout(height=520)
    effectsize_hist_path = os.path.join(out_dir, f"dmr_effect_size_distribution.html")
    # fig_effectsize_hist.write_html(effectsize_hist_path, include_plotlyjs='cdn')
    fig_effectsize_hist.show()
    # print("Saved effect size distribution histogram:", effectsize_hist_path)        

    # plot effect sizes of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh 
    # add color legend names as 'Significant: map_pvalue <= pvalue_thresh' and 'Not Significant: map_pvalue > pvalue_thresh'  
    fig_effectsize_scatter = px.scatter(
        dmr_df,
        x=dmr_df.index,
        y='effect_size',
        color_discrete_map={True: 'red', False: 'blue'},
        color=(dmr_df['map_pvalue'] <= pvalue_thresh), 
        labels={'effect_size':'Effect size (A - B)','index':'Index',
                'color':f'Significant: map_pvalue <= {pvalue_thresh}'},
        title=f"Effect sizes for all CGs (highlighting significant sites with map_pvalue <= {pvalue_thresh})",
    )
    fig_effectsize_scatter.update_layout(height=520)
    effectsize_scatter_path = os.path.join(out_dir, f"dmr_effect_size_scatter.html")
    # fig_effectsize_scatter.write_html(effectsize_scatter_path, include_plotlyjs='cdn')
    fig_effectsize_scatter.show()
    # print("Saved effect size scatter plot:", effectsize_scatter_path)       

    # bar plot effect sizes of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh 
    fig_effectsize_bar = px.bar(
        dmr_df,
        x=dmr_df.index,         
        y='effect_size',
        color=(dmr_df['map_pvalue'] <= pvalue_thresh),
        labels={'effect_size':'Effect size (A - B)','index':'Index',
                'color':f'Significant: map_pvalue <= {pvalue_thresh}'},
        color_discrete_map={True: 'red', False: 'blue'},
        title=f"Effect sizes for all CGs (n={len(dmr_df)}) (highlighting significant sites with map_pvalue <= {pvalue_thresh})",
    )
    fig_effectsize_bar.update_layout(height=520)
    effectsize_bar_path = os.path.join(out_dir, f"dmr_effect_size_bar.html")
    # fig_effectsize_bar.write_html(effectsize_bar_path, include_plotlyjs='cdn')
    fig_effectsize_bar.show()
    # print("Saved effect size bar plot:", effectsize_bar_path) 


    # Bar: effect size per position (without sorting)
    sig['label'] = sig['pos'] + ":" + sig['strand'].astype(str)
    fig_bar_unsorted = px.bar(
        sig,
        x='label',
        y='effect_size',        
        color='effect_size',
        hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        title=f"Effect size for significant CGs (n={len(sig)}) with map_pvalue <= {pvalue_thresh}",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
    fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
    bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
    # fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
    fig_bar_unsorted.show()
    # print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
        

    # Bar: effect size per position (sorted)
    sig_sorted = sig.sort_values('effect_size', ascending=False).copy()
    sig_sorted['label'] = sig_sorted['pos'] + ":" + sig_sorted['strand'].astype(str)
    fig_bar = px.bar(
        sig_sorted,
        x='label',
        y='effect_size',
        color='effect_size',
        hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        title=f"Effect size for significant CGs (n={len(sig_sorted)})",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
    fig_bar.update_layout(xaxis_tickangle=45, height=520)
    bar_path = os.path.join(out_dir, f"dmr_sig_effectsize_p{pvalue_thresh:.3f}.html")
    # fig_bar.write_html(bar_path, include_plotlyjs='cdn')
    fig_bar.show()
    # print("Saved effect-size bar plot:", bar_path)



    # Scatter: sample A vs sample B percent modified (size = total reads, color = effect size)
    fig_scatter = px.scatter(
        sig,
        x='a_perc',
        y='b_perc',
        color='effect_size',
        size='total_reads',
        hover_data=['pos','start','map_pvalue','effect_size','cohen_h'],
        title=f"Significant CGs (map_pvalue <= {pvalue_thresh}) — sample A vs B percent modified",
        labels={'a_perc':'Sample A % modified','b_perc':'Sample B % modified'}
    )
    fig_scatter.update_layout(height=520)
    scatter_path = os.path.join(out_dir, f"dmr_sig_scatter_p{pvalue_thresh:.3f}.html")
    # fig_scatter.write_html(scatter_path, include_plotlyjs='cdn')
    fig_scatter.show()
    # print("Saved scatter plot:", scatter_path)


# TODO: check
- are there really NO diffs between the reads selected with the mC > 70 and mC > 99.5% filtering ??

could be as the mC calles are automatically selected

and in the CRoff the auto threshold 
* in mC > 70 was to 0.79
> Using filter threshold 0.7910156 for C.
* in mC > 99.5 was to  0.79
> Using filter threshold 0.7910156 for C.


and in the Unediter the auto threshold 
* in mC > 70 was to 0.8496
> Using filter threshold 0.8496094 for C.
* in mC > 99.5 was to 0.849
> Using filter threshold 0.8496094 for C.


SAME per condition AUTOMATIC modkit filtering threshold per mC run were set

(NOT 0.995 and not 0.7)